In [ ]:
# Implementation of Density-Induced Ordered Weighted Averaging for Part 4.1 in article (An Illustrative Example)
# M.HosseinZarei-Spring2024

import numpy as np
import pandas as pd
# read Score_excel
Score_excel = pd.read_excel('_Article Example.xlsx')
scores= Score_excel.iloc[:, 1:].values

# to use the first column as index(athlete names)
scores_index = Score_excel.set_index(Score_excel.columns[0]) 

# Create a dictionary : keys are athlete , values are scores
scores = {row: scores_index.loc[row].values for row in scores_index.index}
# print("scores:", scores)

In [2]:
#### Step 1: Determine the density around aki in the set Ak.

# Define the distance function
def distance(a, b):
    return abs(a - b)

# Define the radius r (for neighbors)
r = 0.5

# Calculate density according to equation (22) and (23)
def calculate_density(scores, r):
    densities = np.zeros(len(scores))
    for i, score in enumerate(scores):
        neighbors = [scores[j] for j in range(len(scores)) if distance(score, scores[j]) <= r and i != j]
        if len(neighbors) == 0:
            densities[i] = 0
        else:
            distances = np.array([distance(score, neighbor) for neighbor in neighbors])
            weights = np.exp(-distances)
            weights_sum = np.sum(weights)
            Wk_ij = weights / weights_sum
            density = (1 + np.sum(Wk_ij * distances)) / len(neighbors)
            densities[i] = density ** -1
    return densities

# Calculate densities for each athlete
densities = {athlete: calculate_density(scores[athlete], r) for athlete in scores}
# print("Densities:", densities)

# Export Excel
df = pd.DataFrame(densities).transpose()
df.columns = [f"{i+1}" for i in range(df.shape[1])]
df.to_excel('Densities.xlsx')
df

,1,2,3,4,5,6
A1,0.666667,2.743875,2.436747,0.666667,2.743875,2.590467
A2,1.000000,1.000000,3.000000,3.000000,3.000000,3.000000
A3,2.818466,1.333333,3.503407,3.503407,2.818466,0.000000
A4,1.818182,1.586641,0.666667,1.909304,1.909304,0.909091


In [3]:
#### Step 2: Determine the DIOWA weights by the new maximum orness model 12

# Create bk for decreasing sort "Scores" based on densities according to page (878)
bk = {}
for athlete in densities:
    sorted_indices = np.argsort(-densities[athlete])
    bk[athlete] = scores[athlete][sorted_indices]
# print("bk:", bk)

# Create u_bk for decreasing sort "Densities" based on densities according to page (878)
sorted_scores = {}
u_bk = {}
for athlete in scores:
    sorted_indices = np.argsort(-densities[athlete])
    sorted_scores[athlete] = scores[athlete][sorted_indices]
    u_bk[athlete] = densities[athlete][sorted_indices]
# print("u_bk:", u_bk)

# Calculate f(u(bj)) according to equation (18)
def f_ubj(u_bk):
    return (u_bk - u_bk[-1]) / (u_bk[0] - u_bk[-1])

# Calculate eta1 according to equation (17)
def calculate_eta1(f_ubj, beta):
    n = len(f_ubj)
    sum_f_ubj_squared = np.sum(f_ubj**2)
    sum_f_ubj = np.sum(f_ubj)
    eta1 = -((n * sum_f_ubj_squared - (sum_f_ubj**2)) / (4 * beta))**0.5
    return eta1

# Calculate wj according to equation (16)
def calculate_wj(f_ubj, eta1):
    n = len(f_ubj)
    sum_f_ubj = np.sum(f_ubj)
    wj = (-f_ubj * n / (2 * eta1)) + (sum_f_ubj / (2 * eta1)) + (1 / n)
    wj = np.maximum(wj, 0)
    wj = np.minimum(wj, 1)
    wj /= np.sum(wj)
    return wj

# Calculate k , max_disp , beta according to page (873)
lambda_value = 0.2
def k_calculate(arr):
    first_element = arr[0]
    kvalue = np.sum(arr == first_element)
    return kvalue

DIOWA_weights = {}
for athlete, u_bk_value in u_bk.items():
    n = len(u_bk_value)
    k = k_calculate(u_bk_value)
    max_disp = (1 / (n * k)) - (1 / (n ** 2))
    beta = lambda_value * max_disp
    
    f_u = f_ubj(u_bk_value)
    eta1 = calculate_eta1(f_u, beta)
    wj = calculate_wj(f_u, eta1)
    
    DIOWA_weights[athlete] = wj

# print("DIOWA_weights:", DIOWA_weights)

# Export Excel
df_bk = pd.DataFrame(bk).transpose()
df_u_bk = pd.DataFrame(u_bk).transpose()
df_DIOWA_weights = pd.DataFrame(DIOWA_weights).transpose()

df_bk.columns = [f"{i+1}" for i in range(df_bk.shape[1])]
df_u_bk.columns = [f"{i+1}" for i in range(df_u_bk.shape[1])]
df_DIOWA_weights.columns = [f"{i+1}" for i in range(df_DIOWA_weights.shape[1])]

df_bk.to_excel('Sorted Scores.xlsx')
df_u_bk.to_excel('Sorted Densities.xlsx')
df_DIOWA_weights.to_excel('DIOWA Weights.xlsx')


# display results
print("Sorted Scores:")
display(df_bk)

print("\n\nSorted Densities:")
display(df_u_bk)

print("\n\nDIOWA Weights:")
display(df_DIOWA_weights)


Sorted Scores:


,1,2,3,4,5,6
A1,9.3,9.3,9.4,9.1,8.0,8.5
A2,9.0,9.0,9.0,9.0,8.4,8.4
A3,8.8,8.8,8.7,8.7,9.3,9.9
A4,9.2,9.2,9.3,8.1,8.0,8.6




Sorted Densities:


,1,2,3,4,5,6
A1,2.743875,2.743875,2.590467,2.436747,0.666667,0.666667
A2,3.000000,3.000000,3.000000,3.000000,1.000000,1.000000
A3,3.503407,3.503407,2.818466,2.818466,1.333333,0.000000
A4,1.909304,1.909304,1.818182,1.586641,0.909091,0.666667




DIOWA Weights:


,1,2,3,4,5,6
A1,0.253775,0.253775,0.236401,0.218992,0.018528,0.018528
A2,0.203934,0.203934,0.203934,0.203934,0.092131,0.092131
A3,0.257294,0.257294,0.201865,0.201865,0.081682,0.000000
A4,0.259824,0.259824,0.240548,0.191567,0.048237,0.000000


In [5]:
#### Step 3: Aggregate the six judges’ scores for each athlete using the DIOWA operator according to equation (25)
def aggregate_scores(bk, DIOWA_weights):
    aggregated_scores = {}
    for athlete in bk:
        b = bk[athlete]
        w = DIOWA_weights[athlete]
        aggregated_score = np.sum(w * b)
        aggregated_scores[athlete] = aggregated_score
    return aggregated_scores


final_scores_diowa = aggregate_scores(bk, DIOWA_weights)

# Sort athletes based on final scores in descending order
sorted_athletes = sorted(final_scores_diowa.items(), key=lambda x: x[1], reverse=True)
sorted_final_scores_diowa = {athlete: score for athlete, score in sorted_athletes}

# Export Excel
df = pd.DataFrame(list(sorted_final_scores_diowa.items()), columns=['Athlete', 'Score'])
df.to_excel('Final Result.xlsx', index=False)
df.style.hide(axis='index')      

Athlete,Score
A1,9.240933
A4,8.955446
A2,8.889443
A3,8.800468
